In [1]:
import openai
import pandas as pd
import json
from IPython.display import Markdown
import sys
from tqdm import tqdm

In [2]:
zz=openai.Model.list()

In [3]:
len(zz['data'])

64

In [4]:
model_params = {
    'model': "gpt-4",
    'temperature': 0,
}

In [5]:
def print_save(string, file='../cmp2.md'):
    print(string)
    with open(file,'w') as f:
        f.write(string)

## Load prompt_template.md

In [6]:
sys.path.append('..')
from utils import *
from functools import partial

In [7]:
prompt_dict = partial(load_prompt_template,file='../prompt_template.md')

In [8]:
prompt_dict().keys()

dict_keys(['Preamble', 'Problem-solver', 'Conversation summarizer', 'Construct Kinetic Hamiltonian (continuum version, single-particle)', 'Construct Kinetic Hamiltonian (continuum version, second-quantized)', 'Construct Kinetic Hamiltonian (lattice version)', 'Define each term in Kinetic Hamiltonian (continuum version)', 'Construct Potential Hamiltonian (continuum version)', 'Define each term in Potential Hamiltonian (continuum version)', 'Construct interaction Hamiltonian (real space, lattice version)', 'Construct interaction Hamiltonian (momentum space)', 'Convert from single-particle to second-quantized form, return in matrix', 'Convert from single-particle to second-quantized form, return in summation (expand the matrix)', 'Convert noninteracting Hamiltonian in real space to momentum space (continuum version)', 'Convert noninteracting Hamiltonian in real space to momentum space (lattice version)', 'Convert interacting Hamiltonian in real space to momentum space (lattice version)', 

# Prompt fill place holder

In [9]:
import json

In [2]:
import yaml

In [16]:
with open(f'../Placeholder_questions.yaml','r') as f:
    kwargs_yaml_ph_q = yaml.safe_load(f)

In [3]:
with open(f'{2111.01152}.yaml','r') as f:
    kwargs_yaml = yaml.safe_load(f)

In [30]:
print(': \n  Q: \n  A: \n'.join(kwargs_yaml[13]['placeholder'].keys())+': \n  Q: \n  A: \n')

Hartree_Fock_second_quantized_symbol: 
  Q: 
  A: 
expected_value: 
  Q: 
  A: 
expression_Hartree_1: 
  Q: 
  A: 
expression_Hartree_2: 
  Q: 
  A: 
expression_HF_2: 
  Q: 
  A: 



In [10]:
with open('2111.01152.jsonl','r') as f:
    kwargs_jsonl= [json.loads(line) for line in f]

In [16]:
def load_excerpt(sources):
    excerpt=''
    for tex, lines in sources.items():
        with open(tex,'r') as f:
            f_list=list(f)
            for line in lines:
                excerpt=excerpt+''.join(f_list[line[0]:line[1]])
    return excerpt

In [11]:
prompt_system='''I will provide you a Excerpt of physics paper, and a Template. You will need to fill the placeholders in the template using the correct information from the excerpt.
Here are conventions: 
{..} means a placeholder which you need to fill by extracting information from the excerpt.
{A|B} means you need to make a choice between A and B
[..] means optional sentence. You should decide whether to use it depending on the excerpt.
{{..}} DOES NOT mean a placeholder. You should not change the content inside double curly braces {{..}}.
`You should recall that {expression_second_nonint}.` : this sentence should be kept as is.


Finally, if you cannot figure out the placeholder, you should leave it as is.'''

In [12]:
def drop_text_after(string):
    string_new=''
    for line in string.split('\n'):
        if '===' in line:
            break
        else:
            if 'You should recall that' not in line:
                string_new='\n'.join([string_new,line])
    return string_new[1:]


In [13]:
def extractor(descriptor,message=False):
    sys_msg=[{'role': 'system', 'content': prompt_system}]
    question_prompt='\nTemplate:\n {template} \n\n Excerpt:\n {excerpt}'.format(template=drop_text_after(prompt_dict()[descriptor['task']]), excerpt=load_excerpt(descriptor['source']))
    user_msg=[{'role':'user','content':question_prompt}]
    messages = sys_msg + user_msg
    if message:
        return sys_msg[0]['content']+user_msg[0]['content']
    rs= openai.ChatCompletion.create(messages=messages, **model_params)
    response=rs['choices'][0]['message'].content
    return response


In [20]:
def prompt_template(descriptor):
    return drop_text_after(prompt_dict()[descriptor['task']])

In [21]:
prompt_template(kwargs_jsonl[0])

'You will be instructed to describe the kinetic term of Hamiltonian in {system} in the {real|momentum} space in the {single-particle|second-quantized} form.   \nThe degrees of freedom of the system are: {degrees_of_freedom}.  \nExpress the Kinetic Hamiltonian {kinetic_symbol} using {variable} which are only on the diagonal terms, and arrange the basis in the order of {order}. [Note that the sublattice degrees of freedom is suppressed for now and will be stated later]\n\nUse the following conventions for the symbols:  \n{definition_of_variables}\n\n'

## GPT4

In [73]:
string=''
for kwargs in tqdm(kwargs_jsonl):
    string+='# '+kwargs['task']+'\n'
    response=(extractor(kwargs))
    string+=response+'\n\n'
    
    

100%|██████████| 13/13 [09:14<00:00, 42.63s/it]


In [74]:
print(string)

# Construct Kinetic Hamiltonian (continuum version, single-particle)
You will be instructed to describe the kinetic term of Hamiltonian in a system with two layers in the momentum space in the second-quantized form.   
The degrees of freedom of the system are: valleys, layers, and momentum.  
Express the Kinetic Hamiltonian H_{\tau} using variables which are only on the diagonal terms, and arrange the basis in the order of bottom layer and top layer. Note that the sublattice degrees of freedom is suppressed for now and will be stated later.

Use the following conventions for the symbols:  
$\tau=\pm 1$ represents $\pm K$ valleys, $\bm{\kappa}=\frac{4\pi}{3a_M}\left(1,0\right)$ is at a corner of the moiré Brillouin zone. The $2\times 2$ Hamiltonian hybridizes the bottom layer ($\mathfrak{b}$) and top layer ($\mathfrak{t}$), where the off diagonal terms describe the interlayer tunneling $\Delta_{\text{T},\tau}$, and the diagonal terms describe the momentum-shifted kinetic energy with the

In [76]:
pwd=os.getcwd()

In [77]:
with open(pwd.split('/')[-1]+'_extractor.md','w') as f:
    f.write(string)

In [72]:
print('# '+ kwargs_jsonl[0]['task'])

# Construct Kinetic Hamiltonian (continuum version, single-particle)


In [ ]:
response=(extractor(kwargs_jsonl[0]))

In [ ]:
print(extractor(kwargs_jsonl[0],message=True))

In [ ]:
print(response)

You will be instructed to describe the kinetic term of Hamiltonian in a system with $\pm K$ valleys in the real space in the second-quantized form.   
The degrees of freedom of the system are: the bottom layer ($\mathfrak{b}$) and top layer ($\mathfrak{t}$), and $\pm K$ valleys.  
Express the Kinetic Hamiltonian $H_{\tau}$ using $\bm{k}$ and $\bm{\kappa}$ which are only on the diagonal terms, and arrange the basis in the order of the bottom layer ($\mathfrak{b}$) and top layer ($\mathfrak{t}$). Note that the sublattice degrees of freedom is suppressed for now and will be stated later.

Use the following conventions for the symbols:  
$\tau=\pm 1$ represents $\pm K$ valleys, $\bm{\kappa}=\frac{4\pi}{3a_M}\left(1,0\right)$ is at a corner of the moiré Brillouin zone, the off diagonal terms describe the interlayer tunneling $\Delta_{\text{T},\tau}$, and the diagonal terms describe the momentum-shifted kinetic energy with the effective mass $(m_{\mathfrak{b}},m_{\mathfrak{t}})=(0.65,0.35)m_

In [ ]:
response=(extractor(kwargs_jsonl[1]))

In [33]:
print(kwargs_jsonl[1]['task'])

Define each term in Kinetic Hamiltonian (continuum version)


In [ ]:
print(response)

You will be instructed to construct each term, namely $H_{\tau}$.  
For all energy dispersions, $H_{\tau}$, it characterizes the parabolic dispersion for electrons.   
In addition, an shift of $\tau \bm{\kappa}$ in the momentum $\bm{k}$ for $-\frac{\hbar^2\left(\bm{k}-\tau \bm{\kappa}\right)^2}{2m_\mathfrak{t}}$, respectively.  
You should follow the EXAMPLE below to obtain the correct energy dispersion, select the correct EXAMPLE by noticing the type of dispersion.  
Finally, in the real space, the momentum $\bm{k}=-i \partial_{\bm{r}}$. You should keep the form of $\bm{k}$ in the Hamiltonian for short notations but should remember $\bm{k}$ is an operator.  
Return the expression for $H_{\tau}$ in the Kinetic Hamiltonian, and substitute it into the Kinetic Hamiltonian $\hat{\mathcal{H}}_0$.  

Use the following conventions for the symbols (You should also obey the conventions in all my previous prompts if you encounter undefined symbols. If you find it is never defined or has conflict

In [38]:
print('# '+kwargs_jsonl[2]['task'])

# Construct Potential Hamiltonian (continuum version)


In [ ]:
response=(extractor(kwargs_jsonl[2]))

In [ ]:
print(response)

You will be instructed to describe the potential term of Hamiltonian $H_{\tau}$ in the momentum space in the second-quantized form.  
The potential Hamiltonian has the same degrees of freedom as the kinetic Hamiltonian. The diagonal terms are $-\frac{\hbar^2\bm{k}^2}{2m_\mathfrak{b}}+\Delta_{\mathfrak{b}}(\bm{r})$ and $-\frac{\hbar^2\left(\bm{k}-\tau \bm{\kappa}\right)^2}{2m_\mathfrak{t}}+ \Delta_\mathfrak{t}(\bm{r})+V_{z\mathfrak{t}}$.  
The off-diagonal terms are the coupling between the bottom layer ($\mathfrak{b}$) and top layer ($\mathfrak{t}$), $\Delta_{\text{T},\tau}(\bm{r})$ and $\Delta_{\text{T},\tau}^\dag(\bm{r})$, which should be kept hermitian.  
All other terms are zero.
Express the potential Hamiltonian $H_{\tau}$ using $-\frac{\hbar^2\bm{k}^2}{2m_\mathfrak{b}}+\Delta_{\mathfrak{b}}(\bm{r})$ and $-\frac{\hbar^2\left(\bm{k}-\tau \bm{\kappa}\right)^2}{2m_\mathfrak{t}}+ \Delta_\mathfrak{t}(\bm{r})+V_{z\mathfrak{t}}$ and $\Delta_{\text{T},\tau}(\bm{r})$ and $\Delta_{\text{T},

In [39]:
print('# '+kwargs_jsonl[3]['task'])

# Define each term in Potential Hamiltonian (continuum version)


In [ ]:
response=(extractor(kwargs_jsonl[3]))

In [ ]:
print(response)

You will be instructed to construct each term $H_{\tau}$, namely, $-\frac{\hbar^2\bm{k}^2}{2m_\mathfrak{b}}+\Delta_{\mathfrak{b}}(\bm{r})$, $\Delta_{\text{T},\tau}(\bm{r})$, $\Delta_{\text{T},\tau}^\dag(\bm{r})$, and $-\frac{\hbar^2\left(\bm{k}-\tau \bm{\kappa}\right)^2}{2m_\mathfrak{t}}+ \Delta_\mathfrak{t}(\bm{r})+V_{z\mathfrak{t}}$.  
The expression for diagonal terms are: $-\frac{\hbar^2\bm{k}^2}{2m_\mathfrak{b}}+\Delta_{\mathfrak{b}}(\bm{r})$ and $-\frac{\hbar^2\left(\bm{k}-\tau \bm{\kappa}\right)^2}{2m_\mathfrak{t}}+ \Delta_\mathfrak{t}(\bm{r})+V_{z\mathfrak{t}}$.  
The expression for off-diagonal terms are: $\Delta_{\text{T},\tau}(\bm{r})$ and $\Delta_{\text{T},\tau}^\dag(\bm{r})$.  
Return the expressions for $-\frac{\hbar^2\bm{k}^2}{2m_\mathfrak{b}}+\Delta_{\mathfrak{b}}(\bm{r})$, $\Delta_{\text{T},\tau}(\bm{r})$, $\Delta_{\text{T},\tau}^\dag(\bm{r})$, and $-\frac{\hbar^2\left(\bm{k}-\tau \bm{\kappa}\right)^2}{2m_\mathfrak{t}}+ \Delta_\mathfrak{t}(\bm{r})+V_{z\mathfrak{t}}$, a

In [40]:
print('# '+kwargs_jsonl[4]['task'])

# Convert from single-particle to second-quantized form, return in matrix


In [ ]:
response=(extractor(kwargs[4]))

In [ ]:
print(response)

You will be instructed to construct the second quantized form of the total noninteracting Hamiltonian in the real space.  
The noninteracting Hamiltonian in the real space $\hat{\mathcal{H}}_0$ is the sum of Kinetic Hamiltonian $-\frac{\hbar^2\bm{k}^2}{2m_\mathfrak{b}}$ and Potential Hamiltonian $\Delta_{\mathfrak{b},\tau}(\bm{r})$.  
To construct the second quantized form of a Hamiltonian. You should construct the creation and annihilation operators from the basis explicitly. You should follow the EXAMPLE below to convert a Hamiltonian from the single-particle form to second-quantized form.  
Finally by "total", it means you need to take a summation over the real space position $r$.   
Return the second quantized form of the total noninteracting Hamiltonian $\hat{\mathcal{H}}_0$  

Use the following conventions for the symbols (You should also obey the conventions in all my previous prompts if you encounter undefined symbols. If you find it is never defined or has conflicts in the con

In [42]:
print('# '+kwargs_jsonl[5]['task'])

# Convert from single-particle to second-quantized form, return in summation (expand the matrix)


In [ ]:
response=(extractor(kwargs_jsonl[5]))

In [ ]:
print(response)

You will be instructed to expand the second-quantized form Hamiltonian $\hat{\mathcal{H}}_0$ using $H_{\tau}$ and $\Psi_{\tau}$. You should follow the EXAMPLE below to expand the Hamiltonian.  
You should use any previous knowledge to simplify it. For example, if any term of $H_{\tau}$ is zero, you should remove it from the summation.
Return the expanded form of $\hat{\mathcal{H}}_0$ after simplication.  

Use the following conventions for the symbols (You should also obey the conventions in all my previous prompts if you encounter undefined symbols. If you find it is never defined or has conflicts in the conventions, you should stop and let me know):  
$\hat{\mathcal{H}}_0$ is the second-quantized form Hamiltonian, $H_{\tau}$ is the matrix element, and $\Psi_{\tau}$ is the basis. $\tau=\pm $ represents $\pm K$ valleys, $\hbar \bm{k} = -i \hbar \partial_{\bm{r}}$ is the momentum operator, $\bm{\kappa}=\frac{4\pi}{3a_M}\left(1,0\right)$  is at a corner of the moir\'e Brillouin zone, and

In [43]:
print('# '+kwargs_jsonl[6]['task'])

# Convert noninteracting Hamiltonian in real space to momentum space (continuum version)


In [ ]:
response=(extractor(kwargs_jsonl[6]))

In [ ]:
print_save(response,file='../cmp1.md')

You will be instructed to convert the total noninteracting Hamiltonian in the second quantized form from the basis in real space to the basis by momentum space.  
To do that, you should apply the Fourier transform to $\Psi_{\tau}^\dagger(\bm{r})$ in the real space to $c_{\bm{k}_{\alpha},l_{\alpha},\tau}^\dagger$ in the momentum space, which is defined as $\hat{\mathcal{H}}_0=\sum_{\bm{k}_{\alpha},\bm{k}_{\beta}}\sum_{l_{\alpha},l_{\beta}}\sum_{\tau} h_{\bm{k}_{\alpha}l_{\alpha},\bm{k}_{\beta}l_{\beta}}^{(\tau)} c_{\bm{k}_{\alpha},l_{\alpha},\tau}^\dagger c_{\bm{k}_{\beta},l_{\beta},\tau}$, where $\bm{r}$ is integrated over the entire real space. You should follow the EXAMPLE below to apply the Fourier transform.  
Express the total noninteracting Hamiltonian $\hat{\mathcal{H}}_0$ in terms of $c_{\bm{k}_{\alpha},l_{\alpha},\tau}^\dagger$. Simplify any summation index if possible.  

Use the following conventions for the symbols (You should also obey the conventions in all my previous pr

In [44]:
print('# '+kwargs_jsonl[7]['task'])

# Particle-hole transformation


In [ ]:
response=(extractor(kwargs_jsonl[7]))

In [ ]:
print_save(response,file='../cmp1.md')

You will be instructed to perform a particle-hole transformation.  
Define a hole operator, $b_{\bm{k},l,\tau}$, which equals $c_{\bm{k},l,\tau}^\dagger$.  
You should replace $c_{\bm{k},l,\tau}^\dagger$ with $b_{\bm{k},l,\tau}$, and $c_{\bm{k},l,\tau}$ with $b_{\bm{k},l,\tau}^\dagger$. You should follow the EXAMPLE below to apply the particle-hole transformation.  
Return the $\hat{\mathcal{H}}_0$ in the hole operators.

Use the following conventions for the symbols (You should also obey the conventions in all my previous prompts if you encounter undefined symbols. If you find it is never defined or has conflicts in the conventions, you should stop and let me know):  
The hole operator is defined as $b_{\bm{k},l,\tau}=c_{\bm{k},l,\tau}^\dagger$, and the second non-interacting Hamiltonian in the hole basis is $\hat{\mathcal{H}}_0$.


In [45]:
print('# '+kwargs_jsonl[8]['task'])

# Simplify the Hamiltonian in the particle-hole basis


In [ ]:
response=(extractor(kwargs_jsonl[8]))

In [ ]:
print_save(response,file='../cmp1.md')

You will be instructed to simplify the $\hat{\mathcal{H}}_0$ in the hole basis.  
You should use canonical commutator relation for fermions to reorder the hole operator to the normal order. Normal order means that creation operators always appear before the annihilation operators.  You should follow the EXAMPLE below to simplify it to the normal order.  
Express the $\hat{\mathcal{H}}_0$ in the normal order of $b_{\bm{k},l,\tau}$ and also make $\bm{k}_{\alpha},l_{\alpha},\tau$ always appear before $\bm{k}_{\beta},l_{\beta},\tau$ in the index of $b_{\bm{k},l,\tau}$ and $\hat{\mathcal{H}}_0$.  
Use the following conventions for the symbols (You should also obey the conventions in all my previous prompts if you encounter undefined symbols. If you find it is never defined or has conflicts in the conventions, you should stop and let me know):  
The hole operator is defined as $b_{\bm{k},l,\tau}=c_{\bm{k},l,\tau}^\dagger$, and $\hat{\mathcal{H}}_0$ in the hole basis is defined as $\hat{\math

In [46]:
print('# '+kwargs_jsonl[9]['task'])

# Construct interaction Hamiltonian (momentum space)


In [ ]:
response=(extractor(kwargs_jsonl[9]))

In [ ]:
print_save(response,file='../cmp1.md')

You will be instructed to construct the interaction part of the Hamiltonian $\hat{\mathcal{H}}_{\text{int}}$ in the momentum space.  
The interaction Hamiltonian is a product of four parts.
The first part is the product of four operators with two creation and two annihilation operators following the normal order, namely, creation operators are before annihilation operators. You should follow the order of $l_{\alpha},l_{\beta},l_{\beta},l_{\alpha}$ for the index of operator, and $\bm{k}_{\alpha},\bm{k}_{\beta},\bm{k}_{\gamma},\bm{k}_{\delta}$ for the momentum. 
The second part is the constraint of total momentum conservation, namely the total momentum of all creation operators should be the same as that of all annihilation operators.  
The third part is the interaction form. You should use dual-gate screened Coulomb interaction with $V(\bm{k})= {2\pi e^2\tanh(|\bm{k}|d)}/{(\epsilon |\bm{k}|)}$, where $\bm{k}$ is the transferred total momentum between a creation operator and an annilatio

In [47]:
print('# '+kwargs_jsonl[10]['task'])

# Wick's theorem


In [ ]:
response=(extractor(kwargs_jsonl[10]))

In [ ]:
print_save(response,file='../cmp1.md')

You will be instructed to perform a Hartree-Fock approximation to expand the interaction term, $\hat{\mathcal{H}}_{\text{int}}^{\text{HF}}$.  
You should use Wick's theorem to expand the four-fermion term in $\hat{\mathcal{H}}_{\text{int}}^{\text{HF}}$ into quadratic terms. You should strictly follow the EXAMPLE below to expand using Wick's theorem, select the correct EXAMPLE by noticing the order of four-term product with and without ${{}}^\dagger$, and be extremely cautious about the order of the index and sign before each term.  
You should only preserve the normal terms. Here, the normal terms mean the product of a creation operator and an annihilation operator.  
Return the expanded interaction term after Hartree-Fock approximation as $\hat{\mathcal{H}}^{\text{HF}}$.

Use the following conventions for the symbols (You should also obey the conventions in all my previous prompts if you encounter undefined symbols. If you find it is never defined or has conflicts in the conventions, 

In [48]:
print('# '+kwargs_jsonl[11]['task'])

# Extract quadratic term


In [ ]:
response=(extractor(kwargs_jsonl[11]))

In [ ]:
print_save(response,file='../cmp1.md')

You will be instructed to extract the quadratic terms in the $\hat{\mathcal{H}}^{\text{HF}}$.  
The quadratic terms mean terms that are proportional to $b_{\bm{k}_{\beta},l_{\beta},\tau_{\beta}}^\dagger b_{\bm{k}_{\gamma},l_{\beta},\tau_{\beta}}$ and $b_{\bm{k}_{\beta},l_{\beta},\tau_{\beta}}^\dagger b_{\bm{k}_{\delta},l_{\alpha},\tau_{\alpha}}$, which excludes terms that are solely expectations or products of expectations.  
You should only preserve the quadratic terms in $\hat{\mathcal{H}}^{\text{HF}}$, denoted as $\hat{\mathcal{H}}_{\text{int}}^{\text{HF}}$.  
Return $\hat{\mathcal{H}}_{\text{int}}^{\text{HF}}$.  

Use the following conventions for the symbols (You should also obey the conventions in all my previous prompts if you encounter undefined symbols. If you find it is never defined or has conflicts in the conventions, you should stop and let me know):  
$\hat{\mathcal{H}}^{\text{HF}}$ is the Hartree-Fock Hamiltonian, $\hat{\mathcal{H}}_1$ is the one-body part of the Hamilto

In [49]:
print('# '+kwargs_jsonl[12]['task'])

# Swap the index to combine Hartree and Fock terms


In [ ]:
response=(extractor(kwargs_jsonl[12]))

In [ ]:
print_save(response,file='../cmp1.md')

You will be instructed to simplify the quadratic term $\hat{\mathcal{H}}_{\text{int}}^{\text{HF}}$ through relabeling the index to combine the two Hartree/Fock term into one Hartree/Fock term.  
The logic is that the expected value ($\expval{b_{\bm{k}_{\alpha},l_{\alpha},\tau_{\alpha}}^\dagger b_{\bm{k}_{\delta},l_{\alpha},\tau_{\alpha}}}$ and $\expval{b_{\bm{k}_{\alpha},l_{\alpha},\tau_{\alpha}}^\dagger b_{\bm{k}_{\gamma},l_{\beta},\tau_{\beta}}}$) in the first Hartree term ($b_{\bm{k}_{\beta},l_{\beta},\tau_{\beta}}^\dagger b_{\bm{k}_{\gamma},l_{\beta},\tau_{\beta}}$) has the same form as the quadratic operators in the second Hartree term ($b_{\bm{k}_{\beta},l_{\beta},\tau_{\beta}}^\dagger b_{\bm{k}_{\delta},l_{\alpha},\tau_{\alpha}}$), and vice versa. The same applies to the Fock term.  
This means, if you relabel the index by swapping the index in the "expected value" and "quadratic operators" in the second Hartree term, you can make the second Hartree term look identical to the fi